In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time


In [2]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")
df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [3]:
df.shape

(4026, 7)

In [4]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [6]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffeness Et (MS)',
       'Period (P)'],
      dtype='object')

In [7]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [8]:
y=normalized_df["Period (P)"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

In [21]:

from sklearn.model_selection import KFold,cross_validate
from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import optuna.integration.lightgbm as oplgb
from catboost import CatBoostRegressor

In [43]:
param_grid_simple={'iterations':hp.quniform('iterations',100,1500,50)
                   ,'depth':hp.quniform('depth',3,10,1)
                   ,'l2_leaf_reg':hp.uniform('l2_leaf_reg',0.01,10),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                  }

In [48]:
def hyperopt_objective(params):
    model=CatBoostRegressor(iterations=int(params['iterations'])
              ,depth=int(params['depth'])
              ,l2_leaf_reg=params['l2_leaf_reg']
              ,learning_rate=params['learning_rate']
              ,random_state=99
              )
    
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')
    
    return np.mean(abs(validate_loss['test_score']))

In [49]:


def param_hyperopt(max_evals=100):
    trials=Trials()
    
    early_stop_fn=no_progress_loss(100) 

    params_best=fmin(hyperopt_objective 
                     ,space=param_grid_simple 
                     ,algo=tpe.suggest 
                     ,max_evals=max_evals 
                     ,trials=trials 
                     ,early_stop_fn=early_stop_fn 
                    )
    
    print('best parmas:',params_best)
    return params_best,trials


In [52]:
def hyperopt_validation(params):
    model=CatBoostRegressor(iterations=int(params['iterations'])
              ,depth=int(params['depth'])
              ,l2_leaf_reg=params['l2_leaf_reg']
              ,learning_rate=params['learning_rate']
              ,random_state=99
              )
        
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                  )
    
    return np.mean(abs(validate_loss['test_score']))


In [53]:
# Start time
start_time = time.time()

params_best, trials = param_hyperopt(100)

hyperopt_validation(params_best)

trials.trials[0]

trials.losses()[:10]
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)

100%|██████████| 100/100 [07:44<00:00,  4.64s/trial, best loss: 0.003039172850393704]
best parmas: {'depth': 6.0, 'iterations': 1450.0, 'l2_leaf_reg': 0.3813357607977449, 'learning_rate': 0.15605584736405612}
Total time taken:  469.4890375137329


# CatBoost

In [17]:
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [54]:
from catboost import CatBoostRegressor
params_best['depth'] = int(params_best['depth'])
params_best['iterations'] = int(params_best['iterations'])
cat_reg = CatBoostRegressor(**params_best )
cat_reg.fit(X_train, y_train)

0:	learn: 0.1908680	total: 149ms	remaining: 3m 36s
1:	learn: 0.1665013	total: 154ms	remaining: 1m 51s
2:	learn: 0.1450409	total: 158ms	remaining: 1m 16s
3:	learn: 0.1266998	total: 160ms	remaining: 57.8s
4:	learn: 0.1108983	total: 161ms	remaining: 46.5s
5:	learn: 0.0975579	total: 162ms	remaining: 39s
6:	learn: 0.0857177	total: 163ms	remaining: 33.7s
7:	learn: 0.0761863	total: 164ms	remaining: 29.6s
8:	learn: 0.0682452	total: 165ms	remaining: 26.5s
9:	learn: 0.0605841	total: 166ms	remaining: 23.9s
10:	learn: 0.0544232	total: 168ms	remaining: 21.9s
11:	learn: 0.0487197	total: 168ms	remaining: 20.2s
12:	learn: 0.0435422	total: 169ms	remaining: 18.7s
13:	learn: 0.0397953	total: 170ms	remaining: 17.5s
14:	learn: 0.0356413	total: 172ms	remaining: 16.4s
15:	learn: 0.0326415	total: 173ms	remaining: 15.5s
16:	learn: 0.0298394	total: 174ms	remaining: 14.7s
17:	learn: 0.0274611	total: 175ms	remaining: 13.9s
18:	learn: 0.0254665	total: 176ms	remaining: 13.2s
19:	learn: 0.0238336	total: 177ms	remain

327:	learn: 0.0022279	total: 498ms	remaining: 1.7s
328:	learn: 0.0022060	total: 499ms	remaining: 1.7s
329:	learn: 0.0022039	total: 500ms	remaining: 1.7s
330:	learn: 0.0021999	total: 501ms	remaining: 1.69s
331:	learn: 0.0021958	total: 502ms	remaining: 1.69s
332:	learn: 0.0021917	total: 503ms	remaining: 1.69s
333:	learn: 0.0021868	total: 505ms	remaining: 1.69s
334:	learn: 0.0021838	total: 506ms	remaining: 1.68s
335:	learn: 0.0021812	total: 507ms	remaining: 1.68s
336:	learn: 0.0021780	total: 508ms	remaining: 1.68s
337:	learn: 0.0021748	total: 509ms	remaining: 1.67s
338:	learn: 0.0021723	total: 510ms	remaining: 1.67s
339:	learn: 0.0021656	total: 511ms	remaining: 1.67s
340:	learn: 0.0021620	total: 512ms	remaining: 1.66s
341:	learn: 0.0021508	total: 513ms	remaining: 1.66s
342:	learn: 0.0021479	total: 514ms	remaining: 1.66s
343:	learn: 0.0021445	total: 515ms	remaining: 1.66s
344:	learn: 0.0021422	total: 516ms	remaining: 1.65s
345:	learn: 0.0021384	total: 517ms	remaining: 1.65s
346:	learn: 0.0

500:	learn: 0.0017151	total: 675ms	remaining: 1.28s
501:	learn: 0.0017127	total: 676ms	remaining: 1.28s
502:	learn: 0.0017058	total: 677ms	remaining: 1.27s
503:	learn: 0.0017042	total: 678ms	remaining: 1.27s
504:	learn: 0.0017032	total: 679ms	remaining: 1.27s
505:	learn: 0.0017017	total: 680ms	remaining: 1.27s
506:	learn: 0.0017005	total: 681ms	remaining: 1.27s
507:	learn: 0.0016985	total: 682ms	remaining: 1.26s
508:	learn: 0.0016968	total: 683ms	remaining: 1.26s
509:	learn: 0.0016954	total: 684ms	remaining: 1.26s
510:	learn: 0.0016939	total: 686ms	remaining: 1.26s
511:	learn: 0.0016926	total: 687ms	remaining: 1.26s
512:	learn: 0.0016912	total: 688ms	remaining: 1.26s
513:	learn: 0.0016890	total: 689ms	remaining: 1.25s
514:	learn: 0.0016876	total: 690ms	remaining: 1.25s
515:	learn: 0.0016866	total: 691ms	remaining: 1.25s
516:	learn: 0.0016849	total: 692ms	remaining: 1.25s
517:	learn: 0.0016838	total: 693ms	remaining: 1.25s
518:	learn: 0.0016830	total: 694ms	remaining: 1.25s
519:	learn: 

671:	learn: 0.0014775	total: 859ms	remaining: 995ms
672:	learn: 0.0014768	total: 861ms	remaining: 994ms
673:	learn: 0.0014762	total: 862ms	remaining: 992ms
674:	learn: 0.0014757	total: 863ms	remaining: 991ms
675:	learn: 0.0014749	total: 865ms	remaining: 990ms
676:	learn: 0.0014745	total: 866ms	remaining: 989ms
677:	learn: 0.0014735	total: 867ms	remaining: 987ms
678:	learn: 0.0014671	total: 868ms	remaining: 986ms
679:	learn: 0.0014667	total: 870ms	remaining: 985ms
680:	learn: 0.0014656	total: 872ms	remaining: 984ms
681:	learn: 0.0014648	total: 874ms	remaining: 984ms
682:	learn: 0.0014644	total: 875ms	remaining: 982ms
683:	learn: 0.0014638	total: 876ms	remaining: 981ms
684:	learn: 0.0014628	total: 877ms	remaining: 980ms
685:	learn: 0.0014615	total: 879ms	remaining: 978ms
686:	learn: 0.0014605	total: 880ms	remaining: 977ms
687:	learn: 0.0014598	total: 881ms	remaining: 976ms
688:	learn: 0.0014594	total: 882ms	remaining: 975ms
689:	learn: 0.0014584	total: 884ms	remaining: 973ms
690:	learn: 

955:	learn: 0.0012835	total: 1.23s	remaining: 635ms
956:	learn: 0.0012832	total: 1.23s	remaining: 634ms
957:	learn: 0.0012826	total: 1.23s	remaining: 632ms
958:	learn: 0.0012824	total: 1.23s	remaining: 631ms
959:	learn: 0.0012822	total: 1.23s	remaining: 630ms
960:	learn: 0.0012821	total: 1.23s	remaining: 628ms
961:	learn: 0.0012816	total: 1.24s	remaining: 627ms
962:	learn: 0.0012809	total: 1.24s	remaining: 626ms
963:	learn: 0.0012806	total: 1.24s	remaining: 624ms
964:	learn: 0.0012805	total: 1.24s	remaining: 623ms
965:	learn: 0.0012802	total: 1.24s	remaining: 622ms
966:	learn: 0.0012799	total: 1.24s	remaining: 620ms
967:	learn: 0.0012795	total: 1.24s	remaining: 619ms
968:	learn: 0.0012794	total: 1.25s	remaining: 618ms
969:	learn: 0.0012780	total: 1.25s	remaining: 617ms
970:	learn: 0.0012774	total: 1.25s	remaining: 616ms
971:	learn: 0.0012770	total: 1.25s	remaining: 615ms
972:	learn: 0.0012767	total: 1.25s	remaining: 613ms
973:	learn: 0.0012761	total: 1.25s	remaining: 612ms
974:	learn: 

1122:	learn: 0.0012215	total: 1.42s	remaining: 412ms
1123:	learn: 0.0012205	total: 1.42s	remaining: 411ms
1124:	learn: 0.0012204	total: 1.42s	remaining: 410ms
1125:	learn: 0.0012203	total: 1.42s	remaining: 408ms
1126:	learn: 0.0012199	total: 1.42s	remaining: 407ms
1127:	learn: 0.0012197	total: 1.42s	remaining: 406ms
1128:	learn: 0.0012194	total: 1.42s	remaining: 404ms
1129:	learn: 0.0012193	total: 1.42s	remaining: 403ms
1130:	learn: 0.0012190	total: 1.42s	remaining: 402ms
1131:	learn: 0.0012187	total: 1.43s	remaining: 400ms
1132:	learn: 0.0012185	total: 1.43s	remaining: 399ms
1133:	learn: 0.0012178	total: 1.43s	remaining: 398ms
1134:	learn: 0.0012175	total: 1.43s	remaining: 396ms
1135:	learn: 0.0012172	total: 1.43s	remaining: 395ms
1136:	learn: 0.0012158	total: 1.43s	remaining: 394ms
1137:	learn: 0.0012156	total: 1.43s	remaining: 393ms
1138:	learn: 0.0012153	total: 1.43s	remaining: 391ms
1139:	learn: 0.0012150	total: 1.43s	remaining: 390ms
1140:	learn: 0.0012147	total: 1.43s	remaining:

1435:	learn: 0.0011503	total: 1.77s	remaining: 17.2ms
1436:	learn: 0.0011501	total: 1.77s	remaining: 16ms
1437:	learn: 0.0011500	total: 1.77s	remaining: 14.8ms
1438:	learn: 0.0011499	total: 1.77s	remaining: 13.5ms
1439:	learn: 0.0011497	total: 1.77s	remaining: 12.3ms
1440:	learn: 0.0011497	total: 1.77s	remaining: 11.1ms
1441:	learn: 0.0011495	total: 1.77s	remaining: 9.84ms
1442:	learn: 0.0011495	total: 1.77s	remaining: 8.61ms
1443:	learn: 0.0011492	total: 1.77s	remaining: 7.38ms
1444:	learn: 0.0011492	total: 1.78s	remaining: 6.15ms
1445:	learn: 0.0011491	total: 1.78s	remaining: 4.92ms
1446:	learn: 0.0011489	total: 1.78s	remaining: 3.69ms
1447:	learn: 0.0011488	total: 1.78s	remaining: 2.46ms
1448:	learn: 0.0011486	total: 1.78s	remaining: 1.23ms
1449:	learn: 0.0011485	total: 1.78s	remaining: 0us


In [55]:
y_pred= cat_reg.predict(X_test)
y_pred1=cat_reg.predict(X_val)
y_pred2=cat_reg.predict(X_train)

In [57]:
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [58]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 0.001241207761938778
mean_squared_error: 4.3024056814542115e-06
rmse: 0.0020742241155319286
r2 score: 0.9999137082155393


In [59]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 0.00040652215258932825
mean_squared_error: 1.319017639041971e-06
rmse: 0.001148484932004757
r2 score: 0.9999730346585556


In [60]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 0.001417149823840018
mean_squared_error: 1.4454741390602797e-05
rmse: 0.0038019391618755284
r2 score: 0.9997178498115761
